In [ ]:
#import packages
import os
import glob
from nipype.interfaces.io import DataGrabber, DataSink, SelectFiles # Data i/o
from nipype.interfaces.utility import IdentityInterface     # utility
from nipype.pipeline.engine import Node, Workflow        # pypeline engine
from nipype.interfaces.nipy.preprocess import Trim

from nipype.interfaces.fsl import SliceTimer, FSLCommand, Reorient2Std
import nipype.algorithms.modelgen as model   # model generation
import nipype.algorithms.rapidart as ra      # artifact detection

#set output file ot type for FSL to compressed NIFTI
FSLCommand.set_default_output_type('NIFTI')

# Set study variables
studyhome = '/Volumes/iang/active/BABIES/BABIES_rest/pypelineDev'
raw_data = studyhome + '/raw'
output_dir = studyhome + '/processed/preproc'

subjects_list = ['035-BABIES-T1']#os.listdir(raw_data)
vols_to_trim = 4
interleave = False
TR = 2 # in seconds
slice_dir = 3 # 1=x, 2=y, 3=z
proc_cores = 4 # number of cores of processing for the workflows

In [ ]:
## File handling Nodes

# Identity node- select subjects
infosource = Node(IdentityInterface(fields=['subject_id']),
                     name="infosource")
infosource.iterables = ('subject_id', subjects_list)


# Data grabber- select fMRI and sMRI
templates = {'struct': raw_data + '/{subject_id}/anat/skullstripped_anat.nii',
            'func': raw_data + '/{subject_id}/rest/rest_raw.nii'}
selectfiles = Node(SelectFiles(templates), name='selectfiles')

# Datasink- where our select outputs will go
datasink = Node(DataSink(), name='datasink')
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir

In [ ]:
## Nodes for preprocessing

# Reorient to standard space
reorientfunc = Node(Reorient2Std(), name='reorientfunc')
reorientstruct = Node(Reorient2Std(), name='reorientstruct')

# Trim first 4 volumes
trimvols = Node(Trim(begin_index=vols_to_trim), name='trimvols')

#slice timing correction based on interleaved acquisition
slicetime_correct = Node(SliceTimer(interleaved=interleave, slice_direction=slice_dir,
                                   time_repetition=TR),
                            name='slicetime_correct')

# Motion correction- MEL

# Bias Correction- CC

# Reslice- using MRI_convert 

# Smoothing- LK

# Registration- using ANTs

In [ ]:
## Preprocessing Workflow

# workflowname.connect([(node1,node2,[('node1output','node2input')]),
#                    (node2,node3,[('node2output','node3input')])
#                    ])

preprocwf = Workflow(name='preprocwf')
preprocwf.connect([(infosource,selectfiles,[('subject_id','subject_id')]), 
                   (selectfiles,reorientfunc,[('func','in_file')]),
                   (reorientfunc,trimvols,[('out_file','in_file')]),
                   (trimvols,datasink,[('out_file','trimmed')])
                   #(reorientfunc,datasink,[('out_file','func_reorient')])
                  ])
preprocwf.base_dir = output_dir
preprocwf.write_graph(graph2use='flat')
preprocwf.run('MultiProc', plugin_args={'n_procs': proc_cores})